In [7]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
data = pd.DataFrame.from_csv('song_data.csv', sep=',', index_col=None)

audio_features = pd.DataFrame.from_csv('spotify.csv', sep=',', index_col=None)
audio_features = audio_features.drop(['id', 'analysis_url', 'duration_ms', 'track_href', 'type', 'uri', 'valence'], axis = 1)
audio_features = audio_features.set_index('song').dropna()

#full = data.join(audio_features, on='song')
#full = full.dropna()#[pd.notnull(full['acousticness'])]
#song_matrix = pd.pivot_table(data, index = 'song', columns = 'user_id', values = 'listen_count', fill_value = 0)

In [3]:
n_clusters = 5
kmn = KMeans(n_clusters = n_clusters)
audio_kmn = kmn.fit(audio_features)

In [4]:
clusters = []
for i in range(n_clusters):
    clusters.append(np.asarray([audio_features.index[j] for j in np.where(audio_kmn.labels_ == i)[0]]))

In [12]:
#plt.figure()
#plt.clf()
#plt.scatter(audio_features[:, 0], audio_features[:, 1], c=audio_kmn.predict(audio_features), s=50, cmap='viridis')
clusters[1]#plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

array(['& Down - Boys Noize', '(I Just) Died In Your Arms - Cutting Crew',
       "(The Symphony Of) Blase\\' - Anberlin", ...,
       'mOBSCENE - Marilyn Manson', 're:stacks - Bon Iver',
       'sun drums and soil - Four Tet'],
      dtype='<U159')

In [14]:
song = input()
for i in range(n_clusters):
    if song in clusters[i]:
        index = [i, np.where(clusters[i]==song)[0][0]]
        break

print(index)

selection = pd.DataFrame(index=clusters[index[0]], data={'in': 0})
selection = data.join(selection, on='song').dropna()

cluster_matrix = pd.pivot_table(selection, index = 'song', columns = 'user_id', values = 'listen_count', fill_value = 0)

song_cluster_knn = NearestNeighbors(metric='cosine', algorithm='brute').fit(cluster_matrix)

query = cluster_matrix.index.get_loc(song)
distances, indices = song_cluster_knn.kneighbors(cluster_matrix.iloc[query, :].values.reshape(1, -1), n_neighbors = 6)

for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(cluster_matrix.index[query]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, cluster_matrix.index[indices.flatten()[i]], distances.flatten()[i]))

Through The Wire - Kanye West
[0, 881]
Recommendations for Through The Wire - Kanye West:

1: The Glory - Kanye West, with distance of 0.754036817514376:
2: All Falls Down - Kanye West / Syleena Johnson, with distance of 0.7911657225974366:
3: Homecoming - Kanye West, with distance of 0.8737187025027455:
4: Jesus Walks - Kanye West, with distance of 0.8917534299491273:
5: Take A Minute - K\'Naan, with distance of 0.8942850714233653:
